In [1]:
# This block allows us to import from the benchmark folder,
# as if it was a package installed using pip
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
from attrbench import datasets, evaluation, models
from os import path
import itertools

In [3]:
device = "cpu"
dataset_name = "CIFAR10"
model_name = "resnet20"
download_dataset = False
batch_size = 8
n_batches = 32
target_label = 0
data_root = "../data"
normalize_attrs = True
aggregation_fn = "avg"

In [4]:
patch = torch.load(path.join(data_root, "patches", f"{dataset_name}_{model_name}_patch.pt"))
dataset = datasets.Cifar(batch_size=batch_size, data_location=path.join(data_root, dataset_name))
model = models.CifarResnet(version=model_name, params_loc=path.join(data_root, f"models/{dataset_name}_{model_name}.pth"),
                           output_logits=True)
model.to(device)
model.eval()
pass  # Prevents output from model.eval()

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [5]:
kwargs = {
    "normalize": normalize_attrs,
    "aggregation_fn": aggregation_fn
}

attribution_methods = {
    "GuidedGradCAM": methods.GuidedGradCAM(model, model.get_last_conv_layer(), **kwargs),
    "Gradient": methods.Gradient(model, **kwargs),
    "InputXGradient": methods.InputXGradient(model, **kwargs),
    "IntegratedGradients": methods.IntegratedGradients(model, **kwargs),
    "GuidedBackprop": methods.GuidedBackprop(model, **kwargs),
    "Deconvolution": methods.Deconvolution(model, **kwargs),
    "Random": methods.Random(**kwargs)
}

In [6]:
impact_coverage(data=itertools.islice(dataset.get_dataloader(train=False), n_batches),
                sample_shape=dataset.sample_shape, patch=patch, model=model, methods=attribution_methods,
                device=device, target_label=0)

/home/arne/Documents/PhD/projects/benchmark/venv/lib/python3.6/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:56: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  "Setting backward hooks on ReLU activations."


{'GuidedGradCAM': 0.09466855868639773,
 'Gradient': 0.012786222193228522,
 'InputXGradient': 0.05535964651399924,
 'IntegratedGradients': 0.1036468330134357,
 'GuidedBackprop': 0.09024833434282253,
 'Deconvolution': 0.06663460464956501,
 'Random': 0.05149088199935658}

In [5]:
torch.cuda.is_available()

False